In [ ]:
import os
os.chdir('/content/drive/MyDrive/workspace/Learning_Equality/SVM_DPR_paraphrase-multilingual-mpnet-base-v2-finetuned')

In [ ]:
!pip install cudf-cu11 --extra-index-url=https://pypi.ngc.nvidia.com
!pip install cuml-cu11 --extra-index-url=https://pypi.ngc.nvidia.com

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://pypi.ngc.nvidia.com


In [ ]:
!pip install -U transformers
!pip install sentencepiece
!pip install -U fuzzywuzzy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import lightgbm
import cupy as cp
from cuml.svm import SVC
import cuml
from cuml.preprocessing import MinMaxScaler
from imblearn.under_sampling import RandomUnderSampler
from pathlib import Path
import pickle
import tqdm
import numpy as np
import cudf
import pandas as pd
import gc
import cupy

In [ ]:
class CFG:
    DATA_DIR = Path('./train_data')

In [ ]:
train_df = cudf.read_csv(CFG.DATA_DIR / 'train_df.csv', index_col=0)
train_idx_dict = pickle.load(open(CFG.DATA_DIR / 'train_idx.pkl',mode='rb'))
topics_embedding_cudf = cudf.DataFrame(np.load(CFG.DATA_DIR / 'topics_embedding.npy'))
contents_embedding_cudf = cudf.DataFrame(np.load(CFG.DATA_DIR / 'contents_embedding.npy'))

In [ ]:
train_to_idx = cp.array(train_idx_dict["train_to_idx"])
train_co_idx = cp.array(train_idx_dict["train_co_idx"])

In [ ]:
SAVE_DIR = Path('./svm_model')
for i in range(5):
    SAVE_DIR.mkdir(parents=True, exist_ok=True)
    model_save_path = SAVE_DIR / f'fold_{i}.pkl'
    tr_idx = train_df[train_df.fold!=i].index.to_numpy()
    tr_y = (train_df.label.iloc[tr_idx] * 1).to_numpy()
    val_idx = train_df[train_df.fold==i].index.to_numpy()
    val_y = (train_df.label.iloc[val_idx] * 1)
    rus = RandomUnderSampler(random_state=42)
    tr_idx, tr_y = rus.fit_resample(tr_idx.reshape(-1, 1), tr_y)
    tr_idx = cupy.array(tr_idx)
    val_idx = cupy.array(val_idx)
    print(f'train_num:{len(tr_y)} validation_num:{len(val_y)}')
    tr_idx, tr_y = cupy.array(tr_idx), cupy.array(tr_y)
    tr_idx = tr_idx.reshape(-1)
    tr_X_topic =  topics_embedding_cudf.iloc[train_to_idx[tr_idx]].values
    tr_X_content =  contents_embedding_cudf.iloc[train_co_idx[tr_idx]].values
    tr_X = cp.hstack((tr_X_topic, tr_X_content))
    del tr_X_topic, tr_X_content
    gc.collect()
    if Path(model_save_path).exists():
        print(f'====loading fold:{i}====')
        with open(model_save_path, 'rb') as f:
            pipline = pickle.load(f)
        scaler = pipline['scaler']
        pca = pipline['pca']
        model = pipline['model']
    else:
        print(f'====training fold:{i}====')
        scaler = MinMaxScaler()
        pca = cuml.PCA(n_components=500)
        tr_X = pca.fit_transform(tr_X)
        tr_X = scaler.fit_transform(tr_X)
        model = SVC(kernel='rbf', C=10, gamma=1, verbose=True, probability=True, max_iter=1000)
        model.fit(tr_X, tr_y)
        del tr_X
        gc.collect()
        with open(model_save_path, 'wb') as f:
          pickle.dump({'pca':pca,
                       'scaler':scaler, 
                       'model':model}, f)
    for val_idx_split in tqdm.tqdm(cp.array_split(val_idx, 30), total=30):
      val_X_topic =  topics_embedding_cudf.iloc[train_to_idx[val_idx_split]].values
      val_X_content =  contents_embedding_cudf.iloc[train_co_idx[val_idx_split]].values
      val_X = cp.hstack((val_X_topic, val_X_content))
      val_X = pca.transform(val_X)
      del val_X_topic, val_X_content
      gc.collect()
      val_X = cp.array(scaler.transform(val_X.get()))
      val_pred = model.predict_proba(val_X)
      train_df.loc[val_idx_split, 'predictions_proba'] = val_pred.get()[:,1]
      del val_X
      gc.collect()

train_num:401270 validation_num:12177402
====loading fold:0====


100%|██████████| 30/30 [42:48<00:00, 85.63s/it]


train_num:399110 validation_num:12189909
====training fold:1====
[D] [11:13:42.466183] /project/cpp/src/svm/workingset.cuh:124 Creating working set with 1024 elements
[D] [11:13:42.468708] /project/cpp/src/svm/kernelcache.cuh:133 Allocating kernel tile, size: 1247 MiB
[D] [11:13:42.470184] /project/cpp/src/svm/kernelcache.cuh:138 Allocating x_ws, size: 500 KiB
[D] [11:14:03.536092] /project/cpp/src/svm/smosolver.cuh:185 SMO iteration 500, diff 0.924466
[D] [11:14:24.684750] /project/cpp/src/svm/smosolver.cuh:185 SMO iteration 1000, diff 0.134985
[D] [11:14:24.684869] /project/cpp/src/svm/smosolver.cuh:188 SMO solver finished after 1000 outer iterations, total inner iterations, and diff 0.134985
[D] [11:14:24.696453] /project/python/_skbuild/linux-x86_64-3.8/cmake-build/cuml/common/logger.cxx:3188 Expected column ('F') major order, Converting data, this will result in additional memory utilization.
[D] [11:14:28.485181] /project/python/_skbuild/linux-x86_64-3.8/cmake-build/cuml/common/l

  0%|          | 0/30 [00:00<?, ?it/s]

[D] [11:17:43.143036] /project/python/_skbuild/linux-x86_64-3.8/cmake-build/cuml/common/logger.cxx:3188 Expected column ('F') major order, Converting data, this will result in additional memory utilization.
[D] [11:17:44.627149] /project/python/_skbuild/linux-x86_64-3.8/cmake-build/cuml/common/logger.cxx:3188 Expected column ('F') major order, Converting data, this will result in additional memory utilization.


...

 93%|█████████▎| 28/30 [37:43<02:41, 80.82s/it]

[D] [14:10:33.326432] /project/python/_skbuild/linux-x86_64-3.8/cmake-build/cuml/common/logger.cxx:3188 Expected column ('F') major order, Converting data, this will result in additional memory utilization.
[D] [14:10:34.764928] /project/python/_skbuild/linux-x86_64-3.8/cmake-build/cuml/common/logger.cxx:3188 Expected column ('F') major order, Converting data, this will result in additional memory utilization.


 97%|█████████▋| 29/30 [39:04<01:20, 80.84s/it]

[D] [14:11:54.225559] /project/python/_skbuild/linux-x86_64-3.8/cmake-build/cuml/common/logger.cxx:3188 Expected column ('F') major order, Converting data, this will result in additional memory utilization.
[D] [14:11:55.652195] /project/python/_skbuild/linux-x86_64-3.8/cmake-build/cuml/common/logger.cxx:3188 Expected column ('F') major order, Converting data, this will result in additional memory utilization.


100%|██████████| 30/30 [40:25<00:00, 80.84s/it]


In [ ]:
train_df.to_pandas().to_csv(SAVE_DIR / 'train_df.csv')

In [70]:
def postprocess(df, threshold: float=0.95, top_n: int = 5):
  df.loc[df.predictions_proba>=threshold,'pred'] = 1
  df.loc[df.predictions_proba<threshold,'pred'] = 0 
  result = []
  grouped_df = df.groupby('topic_id')
  for idx, df_ in tqdm.tqdm(grouped_df):
    if df_.pred.sum()==0:
      res_df = df_.iloc[:top_n]
    else:
      res_df = df_[df_.pred==1]
    result.append(res_df.loc[:,['topic_id', 'content_id']])
  result = pd.concat(result, axis=0)
  result = pd.DataFrame(result.groupby('topic_id').apply(lambda x:' '.join(x.content_id)))
  result =result.reset_index().rename(columns={0:'content_ids'})
  result['content_ids'] = result['content_ids'].apply(lambda x:' '.join(x.split(' ')))
  return result

In [71]:
correlations_df = pd.read_csv('../data/row/correlations.csv')

In [72]:
result = postprocess(train_df)

100%|██████████| 61512/61512 [00:57<00:00, 1067.34it/s]


In [73]:
target_df = correlations_df[correlations_df.topic_id.isin(result.topic_id)].reset_index(drop=True)

In [74]:
from sklearn.metrics import fbeta_score
import numpy as np

def calc_f2_score(true_ids, pred_ids):
  true_positives = len(set(true_ids)&set(pred_ids))
  false_positives = len(set(pred_ids)-set(true_ids))
  false_negatives = len(set(true_ids)-set(pred_ids))

  beta = 2
  f2_score = ((1+beta**2)*true_positives)/((1+beta**2)*true_positives + beta**2*false_negatives + false_positives)
  return f2_score

def calc_f2_score_mean(target_df, pred_df):
  shape = target_df.shape
  score = [calc_f2_score(target_df.loc[i, 'content_ids'].split(), pred_df.loc[i, 'content_ids'].split()) for i in range(shape[0])]
  target_df['f2_score'] = score
  return target_df['f2_score'].mean(), target_df

In [75]:
calc_f2_score_mean(target_df, result)

(0.35242676437162374,
              topic_id                                        content_ids  \
 0      t_0008768bdee6       c_34e1424229b4 c_7d1a964d66d5 c_aab93ee667f4   
 1      t_0008a1bd84ba                      c_7ff92a954a3d c_8790b074383e   
 2      t_000d1fb3f2f5  c_07f1d0eec4b2 c_15a6fb858696 c_175e9db3fc44 c...   
 3      t_000feba42136                      c_2bbc650030f4 c_304ee4f59410   
 4      t_00102869fbcb  c_005e793bd0c5 c_066737fa5146 c_08b2070f92e0 c...   
 ...               ...                                                ...   
 61507  t_fff830472691                      c_61fb63326e5d c_8f224e321c87   
 61508  t_fff9e5407d13  c_026db653a269 c_0fb048a6412c c_20de77522603 c...   
 61509  t_fffbe1d5d43c                      c_46f852a49c08 c_6659207b25d5   
 61510  t_fffe14f1be1e                                     c_cece166bad6a   
 61511  t_fffe811a6da9                                     c_92b8fad372ee   
 
        f2_score  
 0      0.272727  
 1      0.0000

In [78]:
result

,topic_id,content_ids
0,t_0008768bdee6,c_563674a55312 c_179302899cfc c_7a656c9cd793 c...
1,t_0008a1bd84ba,c_70cee4e16c16 c_d9113d4c24bc c_cd596a5911d5 c...
2,t_000d1fb3f2f5,c_d7b213393f30 c_371c295721d5 c_55e7244b3fea c...
3,t_000feba42136,c_d59718802908 c_a07f9be390b4 c_5cb131146cbe c...
4,t_00102869fbcb,c_2da287c406c2 c_8559c7cb6343 c_e974407bc1d3 c...
...,...,...
61507,t_fff830472691,c_61fb63326e5d c_0bf565d988de c_773cd43e2ff7
61508,t_fff9e5407d13,c_0fb048a6412c c_15d01758cca7 c_5a80e03b571a c...
61509,t_fffbe1d5d43c,c_0dd3eab0f444
61510,t_fffe14f1be1e,c_47a8fdc3f590 c_f15928e0f771
